In [21]:
"""Task 1. Option Chain Data Collection and BSM Model Application
1. Download the option chain data or manually obtain it from reliable financial websites.
2. Select at least 10 In-The-Money (ITM), 10 Out-Of-The-Money (OTM), and the most appropriate
At-The-Money (ATM) options data for at least two months. This should not be done manually by
looking over data.
3. Use the Black-Scholes-Merton (BSM) model to calculate the fair values of these options and
determine whether they are overpriced or underpriced.
4. Additionally, determine which CE should be purchased and which should not.
Deliverables
Python code for downloading and preprocessing (if required) of option chain data and the
following things in the code itself:
1. Selected ITM, OTM, and ATM options data.
2. Calculated fair values using the BSM model.
3. Assessment of whether each option is overpriced or underpriced."""

'Task 1. Option Chain Data Collection and BSM Model Application\n1. Download the option chain data or manually obtain it from reliable financial websites.\n2. Select at least 10 In-The-Money (ITM), 10 Out-Of-The-Money (OTM), and the most appropriate\nAt-The-Money (ATM) options data for at least two months. This should not be done manually by\nlooking over data.\n3. Use the Black-Scholes-Merton (BSM) model to calculate the fair values of these options and\ndetermine whether they are overpriced or underpriced.\n4. Additionally, determine which CE should be purchased and which should not.\nDeliverables\nPython code for downloading and preprocessing (if required) of option chain data and the\nfollowing things in the code itself:\n1. Selected ITM, OTM, and ATM options data.\n2. Calculated fair values using the BSM model.\n3. Assessment of whether each option is overpriced or underpriced.'

In [22]:
pip install jugaad_data

Note: you may need to restart the kernel to use updated packages.


In [34]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
from scipy.stats import norm
from jugaad_data.nse import NSELive

In [35]:
# Load the CSV file
data = pd.read_csv('t1_data.csv')
print(data.head())

          date   expiryDate  underlyingValue  strikePrice  impliedVolatility  \
0  23-May-2024  27-Jun-2024          22805.3        21000              17.14   
1  23-May-2024  27-Jun-2024          22805.3        21100              16.42   
2  23-May-2024  27-Jun-2024          22805.3        21500              18.84   
3  23-May-2024  27-Jun-2024          22805.3        21600              17.76   
4  23-May-2024  27-Jun-2024          22805.3        21700              17.96   

   lastPrice  bsm_model_price moneyness underpriced/overpriced buy/sell  
0    2060.00              0.0         -                      -        -  
1    1948.40              0.0         -                      -        -  
2    1601.35              0.0         -                      -        -  
3    1503.00              0.0         -                      -        -  
4    1400.00              0.0         -                      -        -  


In [36]:
# Initialize the current price
current_price = data.loc[0, 'underlyingValue']
f = 0
# Ensure the 'moneyness' column exists
data['moneyness'] = None
# Iterate through the data to classify moneyness
for index, row in data.iterrows():
    if row['strikePrice'] < current_price:
        data.loc[index, 'moneyness'] = 'ITM'
    else:
        if f == 0:
            f = 1
            if index == 0:
                data.loc[index, 'moneyness'] = 'ATM'
            elif data.loc[index, 'strikePrice'] - current_price < current_price - data.loc[index-1, 'strikePrice']:
                data.loc[index, 'moneyness'] = 'ATM'
            else:
                data.loc[index, 'moneyness'] = 'OTM'
                data.loc[index-1, 'moneyness'] = 'ATM'
        else:
            data.loc[index, 'moneyness'] = 'OTM'

In [29]:
# Calculate time to expiry and risk-free interest rate
date = data.loc[0, 'date']
expiry_date = data.loc[0, 'expiryDate']
date1 = datetime.datetime.strptime(date, '%d-%b-%Y').date()
date2 = datetime.datetime.strptime(expiry_date, '%d-%b-%Y').date()
difference = date2 - date1
time_to_expiry = difference.days / 365
risk_free_interest_rate = 0.1

In [30]:
import numpy as np
from scipy.stats import norm

def estimated_price(s, v, c=current_price, t=time_to_expiry, r=risk_free_interest_rate, option_type='call'):
    """
    Estimate the price of a European option using the Black-Scholes formula.

    Parameters:
    s (float): Strike price
    v (float): Volatility in percentage
    c (float): Current price of the underlying asset
    t (float): Time to expiry in years
    r (float): Risk-free interest rate
    option_type (str): Type of option ('call' or 'put')

    Returns:
    float: Estimated price of the option
    """
    v /= 100  # Convert percentage volatility to decimal
    d1 = (np.log(c / s) + (r + 0.5 * (v ** 2)) * t) / (v * np.sqrt(t))
    d2 = d1 - v * np.sqrt(t)
    
    if option_type == 'call':
        estimated_price = c * norm.cdf(d1) - s * np.exp(-r * t) * norm.cdf(d2)
    else:
        estimated_price = s * np.exp(-r * t) * norm.cdf(-d2) - c * norm.cdf(-d1)
    
    return round(estimated_price, 2)

# Example usage:
strike_price = 100  # Example strike price
volatility = 20     # Example volatility in percentage
option_price = estimated_price(strike_price, volatility)
print(f"Estimated price: {option_price}")


Estimated price: 22706.25


In [31]:
for index, row in data.iterrows():
    # Calculate BSM model price
    bsm_price = estimated_price(row['strikePrice'], row['impliedVolatility'])
    
    # Assign BSM model price
    data.loc[index, 'bsm_model_price'] = bsm_price
    
    # Determine if underpriced or overpriced
    if bsm_price <= row['lastPrice']:
        data.loc[index, 'underpriced/overpriced'] = 'overpriced'
        data.loc[index, 'buy/sell'] = 'sell'
    else:
        data.loc[index, 'underpriced/overpriced'] = 'underpriced'
        data.loc[index, 'buy/sell'] = 'buy'


In [33]:
# Set display option to show all rows
pd.set_option('display.max_rows', None)

# Print or display your DataFrame 'data'
print(data)  # Or use data to display in a notebook or console


           date   expiryDate  underlyingValue  strikePrice  impliedVolatility  \
0   23-May-2024  27-Jun-2024          22805.3        21000              17.14   
1   23-May-2024  27-Jun-2024          22805.3        21100              16.42   
2   23-May-2024  27-Jun-2024          22805.3        21500              18.84   
3   23-May-2024  27-Jun-2024          22805.3        21600              17.76   
4   23-May-2024  27-Jun-2024          22805.3        21700              17.96   
5   23-May-2024  27-Jun-2024          22805.3        21750              10.11   
6   23-May-2024  27-Jun-2024          22805.3        21800              18.64   
7   23-May-2024  27-Jun-2024          22805.3        21850              18.13   
8   23-May-2024  27-Jun-2024          22805.3        21900              18.58   
9   23-May-2024  27-Jun-2024          22805.3        21950              14.87   
10  23-May-2024  27-Jun-2024          22805.3        22000              19.30   
11  23-May-2024  27-Jun-2024